# **Riesgo de Mercado en Python**
## **Pipeline: Retornos → ARIMA (media) → GARCH (volatilidad condicional) → Forecast → (VaR en Parte 2)**

**Objetivo:** estimar y proyectar volatilidad condicional con enfoque profesional (separación media/varianza) como base para VaR dinámico.

---

### **Mapa del notebook (secuencia profesional)**
1. Gestión de datos y retornos  
2. Estacionariedad y diagnóstico básico  
3. Modelo de media (ARIMA) + residuos  
4. Evidencia ARCH (clustering)  
5. Modelos GARCH/EGARCH/GJR + distribuciones  
6. Diagnóstico del modelo (residuos estandarizados)  
7. Forecast de volatilidad (horizonte H)  
8. Modelamienot VaR y Backtesting



# **1. Gestión de Datos Financieros**


# **2. Series de Tiempo: Estacionariedad y Modelo de Media (ARIMA)**


## BLOQUE 1 — Datos y transformaciones

- Descarga de datos (Close)
- Construcción de **log-precio** y **log-retornos**
- Plots rápidos de sanity check

In [ ]:
# ================================
# BLOQUE 1 — LIBRERÍAS + PARÁMETROS
# ================================
import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Parámetros
ticker = "CHILE.SN"       # CHILE.SN / BSANTANDER.SN / COLBUN.SN / CFINRENTAS.SN / CMPC.SN
start  = "2015-01-01"
end    = None             # None = hasta hoy

import pmdarima as pm


In [ ]:
# ================================
# BLOQUE 1 — DESCARGA + LIMPIEZA
# ================================
df = yf.download(ticker, start=start, end=end, auto_adjust=False, progress=False)

# Normalizar índice (Datetime), ordenar y limpiar
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df.dropna()

df.head()


In [ ]:
# ================================
# BLOQUE 1 — PRECIOS → LOG-RETORNOS
# ================================
# Nota (buenas prácticas):
# Para análisis de retornos/volatilidad/VaR se recomienda usar 'Adj Close'
# (ajustado por dividendos y splits) para evitar saltos artificiales.
USE_ADJ_CLOSE = True
price_col = "Adj Close" if (USE_ADJ_CLOSE and "Adj Close" in df.columns) else "Close"

# Serie de precio
price = df[price_col].copy()
price.name = "price"

# Log-precio y log-retornos
log_price = np.log(price)
log_ret = log_price.diff().dropna()
log_ret.name = "log_ret"

print("Price column:", price_col)
print("Obs price:", price.shape[0])
print("Obs log_ret:", log_ret.shape[0])
log_ret.head()


In [ ]:
# ================================
# BLOQUE 1 — PLOTS (VISUAL RÁPIDO)
# ================================
plt.figure(figsize=(12,4))
plt.plot(price)
plt.title(f"Precio ({price_col}) — {ticker}")
plt.xlabel("Fecha"); plt.ylabel("Precio")
plt.show()

plt.figure(figsize=(12,4))
plt.plot(log_ret)
plt.title(f"Log-retornos — {ticker}")
plt.xlabel("Fecha"); plt.ylabel("log return")
plt.show()


## BLOQUE 1B (Opcional) — Descomposición estacional

Solo para exploración visual; no es requisito para ARIMA financiero en precios.

In [ ]:
# ================================
# DESCOMPOSICIÓN (OPCIONAL)
# ================================
log_price_b = np.log(price.asfreq("B")).ffill().dropna()

decomp = sm.tsa.seasonal_decompose(log_price_b, model="additive", period=252)
decomp.plot()
plt.show()


## BLOQUE 2 — Estacionariedad (ADF)

En finanzas:
- **Precios** suelen NO ser estacionarios (random walk)
- **Retornos** suelen ser más cercanos a estacionarios

Aquí aplicamos ADF para confirmar.

In [ ]:
def adf_test(series: pd.Series, name: str):
    result = adfuller(series.dropna())
    stat, pval = result[0], result[1]
    print(f"\nADF Test: {name}")
    print(f"ADF Statistic : {stat:.4f}")
    print(f"p-value       : {pval:.6f}")
    if pval < 0.05:
        print("→ Serie estacionaria")
    else:
        print("→ Serie NO estacionaria")

adf_test(price, "Precio ({price_col})")
adf_test(log_ret, "Log-retornos")


## BLOQUE 3 — Identificación (ACF / PACF)

Guía visual para sugerir órdenes (p, q) en series estacionarias (usamos **log_ret**).

In [ ]:
plt.figure(figsize=(10,4))
plot_acf(log_ret, lags=20)
plt.title(f"ACF — Log-ret {ticker}")
plt.grid(True)
plt.show()

plt.figure(figsize=(10,4))
plot_pacf(log_ret, lags=20, method="ywm")
plt.title(f"PACF — Log-ret {ticker}")
plt.grid(True)
plt.show()

## BLOQUE 4 — Estimación ARIMA automática (auto_arima) sobre log-ret

Usamos **AIC** para seleccionar (p,d,q) con búsqueda stepwise.

> Nota: modelar en **log-ret** es común por estabilidad numérica y porque el forecast se revierte fácil con `exp()`.

In [ ]:
model_p = pm.auto_arima(
    log_ret,
    start_p=0, start_q=0,
    max_p=5, max_q=5,
    d=None,              # <-- IMPORTANTE: que el modelo encuentre d
    seasonal=False,
    stepwise=True,
    trace=True,
    information_criterion="aic",
    suppress_warnings=True
)

print(model_p.summary())
print("Best order (precio):", model_p.order)



## BLOQUE 5 — Diagnóstico de residuos

Objetivo: que los residuos se comporten como **ruido blanco**.

Hacemos:
- Serie de residuos
- Histograma
- ACF de residuos
- QQ-plot
- Test Ljung-Box (p-value > 0.05 es buena señal)

In [ ]:
residuos = pd.Series(
    model_p.resid(),
    index=log_price.index[-len(model_p.resid()):]
).dropna()


In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.plot(residuos)
plt.title("Residuos ARIMA (log_ret)")

plt.subplot(2,2,2)
plt.hist(residuos, bins=30)
plt.title("Distribución residuos")

plt.subplot(2,2,3)
sm.graphics.tsa.plot_acf(residuos, lags=40, ax=plt.gca())
plt.title("ACF residuos")

plt.subplot(2,2,4)
sm.qqplot(residuos, line='s', ax=plt.gca())
plt.title("QQ plot")

plt.tight_layout()
plt.show()

In [ ]:
lj = acorr_ljungbox(residuos, lags=[10, 20, 30], return_df=True)
lj

# **3. Volatilidad Condicional: ARCH/GARCH**


## Modelamiento Volatilidad Arch - Garch

# BLOQUE 7 — Hechos estilizados: ¿hay clustering de volatilidad?

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# ACF retornos
plt.figure(figsize=(12,4))
plot_acf(log_ret.dropna(), lags=40)
plt.title(f"ACF — log_ret ({ticker})")
plt.grid(True)
plt.show()

# ACF retornos^2 (clustering)
plt.figure(figsize=(12,4))
plot_acf((log_ret.dropna()**2), lags=40)
plt.title(f"ACF — log_ret^2 ({ticker})  (Volatility Clustering)")
plt.grid(True)
plt.show()

# BLOQUE 8 — Test formal: ARCH-LM (¿hay heterocedasticidad condicional?)

In [ ]:
from statsmodels.stats.diagnostic import het_arch

# Opción A: test en retornos
arch_test_ret = het_arch(log_ret.dropna(), nlags=12)
print("ARCH-LM en log_ret")
print(f"LM Stat: {arch_test_ret[0]:.4f} | p-value: {arch_test_ret[1]:.6f}")

# Opción B: test en residuos ARIMA (recomendado si ya modelaste media con ARIMA)
arch_test_res = het_arch(residuos.dropna(), nlags=12)
print("\nARCH-LM en residuos ARIMA")
print(f"LM Stat: {arch_test_res[0]:.4f} | p-value: {arch_test_res[1]:.6f}")


In [ ]:
from arch import arch_model

eps = residuos.dropna()

arch1 = arch_model(
    eps,
    mean="Zero",
    vol="ARCH",
    p=1,
    dist="t"
)

arch1_fit = arch1.fit(disp="off")
print(arch1_fit.summary())

# BLOQUE 9 — Modelo base: GARCH(1,1) sobre residuos del ARIMA (enfoque “profesional”)

In [ ]:
! pip install arch --quiet

In [ ]:
from arch import arch_model

eps = residuos.dropna()

garch11 = arch_model(
    eps,
    mean="Zero",        # porque eps ya son residuos (media ~ 0)
    vol="GARCH",
    p=1, q=1,
    dist="t"            # t-student suele funcionar mejor en finanzas (colas pesadas)
)

garch11_fit = garch11.fit(disp="off")
print(garch11_fit.summary())

# BLOQUE 10 — Extraer y graficar volatilidad condicional (σ_t)

In [ ]:
import matplotlib.pyplot as plt

cond_vol = garch11_fit.conditional_volatility  # sigma_t

plt.figure(figsize=(12,4))
plt.plot(cond_vol)
plt.title(f"Volatilidad condicional σ_t — GARCH(1,1) ({ticker})")
plt.xlabel("Fecha"); plt.ylabel("σ_t")
plt.show()


# BLOQUE 11 — Comparación: volatilidad condicional vs histórica (rolling)

In [ ]:
# Vol histórica rolling (21 días hábiles), sobre retornos (no residuos)
vol_hist = log_ret.dropna().rolling(21).std()

plt.figure(figsize=(12,4))
plt.plot(vol_hist, label="Vol histórica (rolling 21)")
plt.plot(cond_vol, label="Vol condicional (GARCH)", alpha=0.8)
plt.title(f"Volatilidad: histórica vs condicional — {ticker}")
plt.xlabel("Fecha"); plt.ylabel("Vol")
plt.legend()
plt.show()


# BLOQUE 12 — Diagnóstico rápido del GARCH (residuos estandarizados)

In [ ]:

# Residuo estandarizado z_t = eps_t / sigma_t
std_resid = garch11_fit.std_resid

# ACF de z_t^2 (idealmente más limpio)
plt.figure(figsize=(12,4))
plot_acf((std_resid**2), lags=40)
plt.title("ACF — residuos estandarizados^2 (post-GARCH)")
plt.show()

# Ljung-Box en z_t^2
lb = acorr_ljungbox(std_resid**2, lags=[10, 20, 30], return_df=True)
lb

# BLOQUE 13 — Modelos de Volatilidad Asimétrica: EGARCH y GJR-GARCH (Introspección + Comparación)
🎯 Objetivo del bloque

Explorar alternativas a GARCH(1,1) cuando sospechamos que el mercado reacciona distinto ante noticias malas vs buenas (leverage effect), y comparar modelos con AIC/BIC + diagnóstico.

🅰 Punto A — EGARCH(1,1)
¿De qué trata?

EGARCH (Exponential GARCH) modela la volatilidad en escala logarítmica.

Diferencias clave vs GARCH

✅ Permite asimetría con γ (leverage effect)
✅ Asegura σ² > 0 automáticamente (porque modela log σ²)
✅ Maneja mejor colas y volatilidad extrema en algunos activos

Interpretación del parámetro γ:

γ < 0 (típico en equity): shocks negativos aumentan más la vol que shocks positivos.

In [ ]:
from arch import arch_model
import matplotlib.pyplot as plt

eps = residuos.dropna()

# EGARCH(1,1)
model_egarch = arch_model(
    eps,
    mean="Zero",
    vol="EGARCH",
    p=1, q=1,
    dist="t"   # colas pesadas típicas en finanzas
)

res_egarch = model_egarch.fit(disp="off")
print(res_egarch.summary())

# Volatilidad condicional ajustada
vol_egarch = res_egarch.conditional_volatility

plt.figure(figsize=(12,4))
plt.plot(vol_egarch)
plt.title(f"Volatilidad Condicional — EGARCH(1,1) ({ticker})")
plt.xlabel("Fecha"); plt.ylabel("σ_t")
plt.show()


🅱 Punto B — GJR-GARCH(1,1)
¿De qué trata?

GJR-GARCH agrega un término que se activa si el shock fue negativo.

Diferencias clave vs GARCH

✅ Asimetría explícita: si ε<0, el shock pesa más
✅ Muy usado en equity y riesgo porque es intuitivo
🔸 Requiere restricciones para asegurar positividad/estabilidad

Interpretación del parámetro γ (en GJR):

γ > 0: shocks negativos aumentan más la vol (leverage effect)

In [ ]:
from arch import arch_model
import matplotlib.pyplot as plt

eps = residuos.dropna()

# GJR-GARCH(1,1): en arch se implementa como GARCH con o=1 (término asimétrico)
model_gjr = arch_model(
    eps,
    mean="Zero",
    vol="GARCH",
    p=1, o=1, q=1,
    dist="t"
)

res_gjr = model_gjr.fit(disp="off")
print(res_gjr.summary())

# Volatilidad condicional ajustada
vol_gjr = res_gjr.conditional_volatility

plt.figure(figsize=(12,4))
plt.plot(vol_gjr)
plt.title(f"Volatilidad Condicional — GJR-GARCH(1,1) ({ticker})")
plt.xlabel("Fecha"); plt.ylabel("σ_t")
plt.show()


# BLOQUE 13.1 — Graficos de distintas volatilidades

In [ ]:
from arch import arch_model

# === Funciones estilo profe pero para eps (residuos ARIMA) ===
def fit_garch(eps, dist='normal'):
    model = arch_model(eps, mean="Zero", vol='GARCH', p=1, q=1, dist=dist)
    results = model.fit(disp='off')
    return results

def fit_egarch(eps, dist='normal'):
    model = arch_model(eps, mean="Zero", vol='EGARCH', p=1, q=1, dist=dist)
    results = model.fit(disp='off')
    return results

def fit_gjr_garch(eps, dist='normal'):
    model = arch_model(eps, mean="Zero", vol='GARCH', p=1, o=1, q=1, dist=dist)
    results = model.fit(disp='off')
    return results

#########################################################################################
import pandas as pd
import matplotlib.pyplot as plt

eps = residuos.dropna()

# 1) Ajuste con Normal
res_garch_n  = fit_garch(eps, dist='normal')
res_egarch_n = fit_egarch(eps, dist='normal')
res_gjr_n    = fit_gjr_garch(eps, dist='normal')

# 2) Ajuste con t-student
res_garch_t  = fit_garch(eps, dist='t')
res_egarch_t = fit_egarch(eps, dist='t')
res_gjr_t    = fit_gjr_garch(eps, dist='t')

# 3) Ajuste con skewt (asimétrica)
res_garch_s  = fit_garch(eps, dist='skewt')
res_egarch_s = fit_egarch(eps, dist='skewt')
res_gjr_s    = fit_gjr_garch(eps, dist='skewt')

########################################################################################

def plot_vols(eps, res_garch, res_egarch, res_gjr, title=""):
    df = pd.DataFrame({
        "eps": eps,
        "GARCH(1,1)": res_garch.conditional_volatility,
        "EGARCH(1,1)": res_egarch.conditional_volatility,
        "GJR-GARCH(1,1)": res_gjr.conditional_volatility
    })

    plt.figure(figsize=(8,5))
    plt.plot(df["eps"], label="eps (residuos)", color="gray", alpha=0.6)
    plt.plot(df["GARCH(1,1)"], label="GARCH(1,1)", alpha=0.8)
    plt.plot(df["EGARCH(1,1)"], label="EGARCH(1,1)", alpha=0.8)
    plt.plot(df["GJR-GARCH(1,1)"], label="GJR-GARCH(1,1)", alpha=0.8)
    plt.title(title)
    plt.xlabel("Tiempo")
    plt.ylabel("Residuo / Volatilidad")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

#########################################################################################

plot_vols(eps, res_garch_n, res_egarch_n, res_gjr_n, title="Volatilidad estimada (Normal)")
plot_vols(eps, res_garch_t, res_egarch_t, res_gjr_t, title="Volatilidad estimada (t-student)")
plot_vols(eps, res_garch_s, res_egarch_s, res_gjr_s, title="Volatilidad estimada (skewt)")


# BLOQUE 14 — Forecast de Volatilidad

In [ ]:
h = 20
n_sim = 5000  # puedes bajar a 1000 si va lento

# Forecast (multi-step) usando simulación
fcast_garch  = garch11_fit.forecast(horizon=h, method="simulation", simulations=n_sim, reindex=False)
fcast_egarch = res_egarch.forecast(horizon=h, method="simulation", simulations=n_sim, reindex=False)
fcast_gjr    = res_gjr.forecast(horizon=h, method="simulation", simulations=n_sim, reindex=False)

# Extraer sigma forecast: sqrt(var) de la última fila
sigma_garch  = np.sqrt(fcast_garch.variance.values[-1])
sigma_egarch = np.sqrt(fcast_egarch.variance.values[-1])
sigma_gjr    = np.sqrt(fcast_gjr.variance.values[-1])

df_forecast = pd.DataFrame({
    "h": np.arange(1, h+1),
    "GARCH(1,1)": sigma_garch,
    "EGARCH(1,1)": sigma_egarch,
    "GJR-GARCH(1,1)": sigma_gjr
})

display(df_forecast)

plt.figure(figsize=(8,5))
plt.plot(df_forecast["h"], df_forecast["GARCH(1,1)"], marker="o", label="GARCH")
plt.plot(df_forecast["h"], df_forecast["EGARCH(1,1)"], marker="o", label="EGARCH")
plt.plot(df_forecast["h"], df_forecast["GJR-GARCH(1,1)"], marker="o", label="GJR-GARCH")

plt.title(f"Forecast de Volatilidad (σ) — {h} días (simulation)")
plt.xlabel("Horizonte (días)")
plt.ylabel("Sigma forecast")
plt.legend()
plt.grid(True)
plt.show()

# BLOQUE 15 — Value at Risk (VaR): Enfoques Comparativos

El Value at Risk (VaR) mide la pérdida máxima esperada en un horizonte dado
con un nivel de confianza determinado.

En este bloque implementaremos tres enfoques clásicos:

1. VaR Paramétrico (Modelo condicional ARIMA + GARCH)
2. VaR Histórico
3. VaR Monte Carlo
4. Extensión: Monte Carlo Multivariado (Cholesky)

El objetivo es comparar sus fundamentos, supuestos y diferencias.

## 15.1 VaR Paramétrico Condicional

Se basa en:

VaR_t = μ̂_t + q_α · σ̂_t

Ventajas:
- Captura volatilidad dinámica
- Permite colas pesadas (t-student)

Desventajas:
- Depende del modelo
- Sensible a mala especificación

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, t

# ----------------------------
# 1) Definir el índice "válido" para VaR
# ----------------------------
# En este notebook, el GARCH se ajusta sobre eps = residuos.dropna()
# -> su volatilidad condicional (sigma_t) está indexada igual que eps.
# Para comparar r_t vs VaR_t de forma correcta, trabajaremos sobre ese mismo índice.
try:
    idx = residuos.dropna().index
except Exception:
    idx = log_ret.dropna().index

# ----------------------------
# 2) Base dataframe con retornos y media condicional
# ----------------------------
df_var = pd.DataFrame(index=idx)

# Retornos observados (alineados al mismo índice)
df_var["r"] = log_ret.reindex(idx).astype(float)

# Media condicional (ARIMA). Si no existe, asumimos 0 (muy común en práctica).
try:
    df_var["mu"] = arima_res.fittedvalues.reindex(idx).astype(float)
except Exception:
    df_var["mu"] = 0.0

df_var["mu"] = df_var["mu"].fillna(0.0)

# ----------------------------
# 3) Volatilidad condicional: modelo seleccionado
# ----------------------------
# Modelo final (puedes cambiar a res_garch_t o res_egarch_t si quieres)
garch_res = res_gjr_t

# sigma_t (ya viene alineada al índice de eps)
sigma_series = pd.Series(garch_res.conditional_volatility, index=idx, name="sigma")
df_var["sigma"] = sigma_series

# Limpieza final
df_var = df_var.dropna(subset=["r", "mu", "sigma"]).copy()

# ----------------------------
# 4) Cuantiles: t-Student estandarizada a var=1
# ----------------------------
# Nota: la t estándar tiene Var = nu/(nu-2). Para shocks con Var=1:
# q_std = t_ppf(alpha, nu) * sqrt((nu-2)/nu)
if "nu" not in garch_res.params.index:
    raise KeyError("No encontré 'nu' en garch_res.params. Revisa garch_res.params.index")

nu = float(garch_res.params["nu"])

def q_student_std(alpha: float) -> float:
    return t.ppf(alpha, df=nu) * np.sqrt((nu - 2) / nu)

alphas = {"VaR_95": 0.05, "VaR_99": 0.01}

for name, a in alphas.items():
    df_var[name] = df_var["mu"] + q_student_std(a) * df_var["sigma"]

# ----------------------------
# 5) Gráfico (retornos vs VaR)
# ----------------------------
plt.figure(figsize=(12,5))
plt.plot(df_var.index, df_var["r"], label="Retornos", alpha=0.6)
plt.plot(df_var.index, df_var["VaR_95"], label="VaR 95%")
plt.plot(df_var.index, df_var["VaR_99"], label="VaR 99%")
plt.title("VaR Paramétrico Dinámico — GJR-GARCH (t)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

df_var.tail()


## 15.2 VaR Histórico

El VaR histórico no asume distribución paramétrica.

Se calcula directamente como el percentil empírico
de los retornos pasados.

VaR_α = Percentil_α(r_t)

In [ ]:
# VaR histórico global
var_hist_95 = np.percentile(log_ret.dropna(), 5)
var_hist_99 = np.percentile(log_ret.dropna(), 1)

print("VaR Histórico 95%:", var_hist_95)
print("VaR Histórico 99%:", var_hist_99)

# Rolling 250 días (~1 año bursátil)
window = 250

df_var["VaR_hist_95"] = df_var["r"].rolling(window).quantile(0.05)
df_var["VaR_hist_99"] = df_var["r"].rolling(window).quantile(0.01)

df_var[["r", "VaR_hist_95", "VaR_hist_99"]].dropna().tail()

plt.figure(figsize=(13,6))

plt.plot(df_var.index, df_var["r"], 
         color="black", alpha=0.5, label="Retornos")

plt.plot(df_var.index, df_var["VaR_hist_95"], 
         color="blue", linewidth=1.5, label="VaR Hist 95%")

plt.plot(df_var.index, df_var["VaR_hist_99"], 
         color="purple", linewidth=1.5, label="VaR Hist 99%")

plt.title("VaR Histórico Rolling (250 días)", fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## 15.3 VaR Monte Carlo (Univariado)

Se simulan N escenarios para el retorno futuro:

r_{t+1} = μ̂_t + σ̂_t · ε

donde ε sigue la distribución estimada (normal o t).

El VaR se calcula como el percentil de la distribución simulada.

In [ ]:
# ==============================
# Monte Carlo VaR (1 día)
# ==============================
N = 20000

last_mu = df_var["mu"].iloc[-1]
last_sigma = df_var["sigma"].iloc[-1]
nu = garch_res.params["nu"]

# shocks t estandarizados
shocks = t.rvs(df=nu, size=N) * np.sqrt((nu-2)/nu)

sim_returns = last_mu + last_sigma * shocks

mc_var_95 = np.percentile(sim_returns, 5)
mc_var_99 = np.percentile(sim_returns, 1)

plt.figure(figsize=(10,5))
plt.hist(sim_returns, bins=100, density=True, alpha=0.6)
plt.axvline(mc_var_95, color="orange", label="MC VaR 95%")
plt.axvline(mc_var_99, color="red", label="MC VaR 99%")

plt.title("Monte Carlo — Distribución Simulada Retorno 1 día")
plt.legend()
plt.show()

print("Monte Carlo VaR 95%:", mc_var_95)
print("Monte Carlo VaR 99%:", mc_var_99)

## 15.4 Monte Carlo Multivariado (Cholesky)

Cuando trabajamos con portafolios, necesitamos simular
retornos correlacionados.

Si Σ es la matriz de covarianza, podemos usar
la descomposición de Cholesky:

Σ = L L'

Simulamos:

Z ~ N(0,I)
ε = L Z

Así obtenemos shocks correlacionados.

In [ ]:
# ==============================
# Monte Carlo Multivariado (Ejemplo)
# ==============================

# Supongamos retornos de dos activos
returns_matrix = pd.concat([log_ret, log_ret.shift(1)], axis=1).dropna()
returns_matrix.columns = ["Asset1", "Asset2"]

cov_matrix = returns_matrix.cov().values
L = np.linalg.cholesky(cov_matrix)

N = 20000
Z = np.random.normal(size=(2, N))
correlated_shocks = L @ Z

sim_portfolio = correlated_shocks.sum(axis=0)

portfolio_var_95 = np.percentile(sim_portfolio, 5)

plt.figure(figsize=(10,5))
plt.hist(sim_portfolio, bins=100, density=True, alpha=0.6)
plt.axvline(portfolio_var_95, color="red", label="Portfolio VaR 95%")

plt.title("Monte Carlo Multivariado (Cholesky)")
plt.legend()
plt.show()

print("Portfolio VaR 95%:", portfolio_var_95)

## 15.5 Comparación de Enfoques

| Método        | Supuestos | Dinámico | Pros | Contras |
|--------------|----------|----------|------|---------|
| Paramétrico  | Sí       | Sí       | Flexible | Depende del modelo |
| Histórico    | No       | No       | Simple | No captura cambios estructurales |
| Monte Carlo  | Sí       | Sí       | Muy flexible | Computacionalmente intensivo |

En la práctica bancaria, el VaR paramétrico condicional
es común en market risk, mientras que Monte Carlo
es estándar para portafolios complejos.

# BLOQUE 16 — Backtesting del VaR (Validación Cuantitativa)

## 16.1 ¿Qué es backtesting y por qué se hace?

El VaR es una predicción probabilística:
- VaR 99% significa que esperamos violaciones ~1% de los días.
- VaR 95% significa violaciones ~5% de los días.

**Backtesting** es comparar el VaR calculado con los retornos realmente observados para verificar si el modelo:

1) **Tiene la cobertura correcta** (frecuencia de violaciones ≈ α)  
2) **No tiene clustering de violaciones** (violaciones independientes en el tiempo)

En la práctica cuant (bancos/AFPs/mesas), backtesting responde:
- ¿El VaR subestima o sobreestima riesgo?
- ¿Las violaciones ocurren de forma aleatoria (como debería) o se agrupan (modelo mal especificado)?
- ¿El modelo reacciona suficientemente rápido en shocks?

---

## 16.2 Qué buscamos (criterios profesionales)

### A) Cobertura incondicional (Kupiec)
Evalúa si el porcentaje observado de violaciones coincide con el porcentaje esperado α.

- H0: P(violación) = α  
- Si rechazamos H0 → VaR está mal calibrado (muy conservador o subestima riesgo)

### B) Independencia de violaciones (Christoffersen)
Evalúa si las violaciones son independientes (no deberían agruparse).

- H0: las violaciones no dependen de la violación previa  
- Si rechazamos H0 → hay **clustering** (típico cuando la volatilidad cambia rápido y el modelo no la capta)

### C) Cobertura condicional (Christoffersen completo)
Combina ambas:
- Cobertura correcta + Independencia correcta

---

## 16.3 Interpretación cuant (cómo lo usan en la realidad)

Un VaR "bueno" típicamente:
- No rechaza Kupiec (p-value alto)
- No rechaza Independencia (p-value alto)
- No rechaza Cobertura condicional

Además se revisa:
- Número esperado vs observado de violaciones
- Gráfico de violaciones en el tiempo
- Si las violaciones se concentran en crisis (normal) pero sin clustering excesivo

> Nota: Esto es el corazón del control de modelos (Model Risk / Market Risk Validation).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2

# ============================================================
# 16.0 Helpers
# ============================================================

def make_hits(df, var_col, r_col="r"):
    """
    Hit = 1 si hay violación (retorno < VaR), 0 si no.
    Devuelve serie alineada y sin NaN.
    """
    tmp = df[[r_col, var_col]].dropna().copy()
    hit = (tmp[r_col] < tmp[var_col]).astype(int)
    hit.name = f"hit_{var_col}"
    return hit

def kupiec_test(hit, alpha):
    """
    Kupiec (Unconditional Coverage) LRuc test.
    H0: P(hit=1) = alpha
    """
    x = int(hit.sum())
    T = int(hit.count())
    phat = x / T if T > 0 else np.nan

    # evitar logs inválidos si x=0 o x=T
    eps = 1e-12
    phat_c = np.clip(phat, eps, 1 - eps)
    alpha_c = np.clip(alpha, eps, 1 - eps)

    # log-likelihoods
    ll_null = x * np.log(alpha_c) + (T - x) * np.log(1 - alpha_c)
    ll_alt  = x * np.log(phat_c)  + (T - x) * np.log(1 - phat_c)

    LRuc = -2 * (ll_null - ll_alt)
    pval = 1 - chi2.cdf(LRuc, df=1)

    return {"T": T, "x": x, "phat": phat, "LRuc": LRuc, "pvalue": pval}

def christoffersen_independence_test(hit):
    """
    Christoffersen independence test (LRind).
    H0: hits independientes (Markov de orden 0)
    Se basa en conteos de transiciones:
    N00, N01, N10, N11
    """
    h = hit.dropna().astype(int).values
    if len(h) < 2:
        return {"N00": np.nan, "N01": np.nan, "N10": np.nan, "N11": np.nan,
                "LRind": np.nan, "pvalue": np.nan}

    h_lag = h[:-1]
    h_cur = h[1:]

    N00 = int(((h_lag == 0) & (h_cur == 0)).sum())
    N01 = int(((h_lag == 0) & (h_cur == 1)).sum())
    N10 = int(((h_lag == 1) & (h_cur == 0)).sum())
    N11 = int(((h_lag == 1) & (h_cur == 1)).sum())

    # probabilidades condicionales
    eps = 1e-12
    pi01 = N01 / (N00 + N01) if (N00 + N01) > 0 else 0.0
    pi11 = N11 / (N10 + N11) if (N10 + N11) > 0 else 0.0

    # probabilidad incondicional
    pi = (N01 + N11) / (N00 + N01 + N10 + N11) if (N00 + N01 + N10 + N11) > 0 else 0.0

    # clip para logs
    pi01 = float(np.clip(pi01, eps, 1 - eps))
    pi11 = float(np.clip(pi11, eps, 1 - eps))
    pi   = float(np.clip(pi,   eps, 1 - eps))

    # log-likelihoods
    ll_ind = (N00 * np.log(1 - pi01) + N01 * np.log(pi01) +
              N10 * np.log(1 - pi11) + N11 * np.log(pi11))

    ll_iid = ((N00 + N10) * np.log(1 - pi) + (N01 + N11) * np.log(pi))

    LRind = -2 * (ll_iid - ll_ind)
    pval = 1 - chi2.cdf(LRind, df=1)

    return {"N00": N00, "N01": N01, "N10": N10, "N11": N11, "LRind": LRind, "pvalue": pval}

def christoffersen_conditional_coverage(hit, alpha):
    """
    Christoffersen conditional coverage (LRcc) = LRuc + LRind
    """
    uc = kupiec_test(hit, alpha)
    ind = christoffersen_independence_test(hit)

    LRcc = uc["LRuc"] + ind["LRind"]
    pval = 1 - chi2.cdf(LRcc, df=2)

    return {"LRcc": LRcc, "pvalue": pval, "uc": uc, "ind": ind}

# ============================================================
# 16.1 Backtesting para VaR 95% y 99%
# ============================================================

tests = {}

for var_col, alpha in [("VaR_95", 0.05), ("VaR_99", 0.01)]:
    hit = make_hits(df_var, var_col=var_col, r_col="r")
    cc = christoffersen_conditional_coverage(hit, alpha)

    tests[var_col] = {
        "alpha": alpha,
        "T": cc["uc"]["T"],
        "x": cc["uc"]["x"],
        "phat": cc["uc"]["phat"],
        "LRuc": cc["uc"]["LRuc"],
        "p_uc": cc["uc"]["pvalue"],
        "LRind": cc["ind"]["LRind"],
        "p_ind": cc["ind"]["pvalue"],
        "LRcc": cc["LRcc"],
        "p_cc": cc["pvalue"],
    }

results_bt = pd.DataFrame(tests).T
results_bt

In [ ]:
# ============================================================
# 16.2 Gráfico de violaciones (hits) para VaR 99%
# ============================================================

hit_99 = make_hits(df_var, "VaR_99", "r")
tmp_99 = df_var[["r", "VaR_99"]].dropna().copy()
viol_99 = tmp_99[tmp_99["r"] < tmp_99["VaR_99"]]

plt.figure(figsize=(13,6))
plt.plot(tmp_99.index, tmp_99["r"], label="Retornos", alpha=0.6, linewidth=1)
plt.plot(tmp_99.index, tmp_99["VaR_99"], label="VaR 99%", linewidth=1)
plt.scatter(viol_99.index, viol_99["r"], label="Violaciones 99%", s=15)
plt.title("Backtesting visual — Violaciones VaR 99%")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# ============================================================
# 16.3 Rolling hit rate (¿se acerca a 1% o 5%?)
# ============================================================

window = 250  # 1 año bursátil aprox

hit_95 = make_hits(df_var, "VaR_95", "r")

roll_95 = hit_95.rolling(window).mean()
roll_99 = hit_99.rolling(window).mean()

plt.figure(figsize=(13,5))
plt.plot(roll_95.index, roll_95.values, label="Rolling hit rate VaR 95% (esperado 5%)")
plt.plot(roll_99.index, roll_99.values, label="Rolling hit rate VaR 99% (esperado 1%)")
plt.title(f"Rolling hit rate ({window} días)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# ============================================================
# 16.4 Tabla simple: esperado vs observado
# ============================================================

for var_col, alpha in [("VaR_95", 0.05), ("VaR_99", 0.01)]:
    hit = make_hits(df_var, var_col)
    T = hit.count()
    x = hit.sum()
    print(f"{var_col}: Observadas={x} de {T} | Tasa={x/T:.4f} | Esperada={alpha:.4f}")

# **Apéndice A — Forecast de Precio con ARIMA (opcional)**
Este bloque no es central para VaR, pero se conserva como referencia exploratoria.


# BLOQUE 6 (Prueba) — Forecast de precio (ARIMA sobre log_price)

In [ ]:
log_price = log_price.asfreq("B").ffill()
price = price.asfreq("B").ffill()
model = ARIMA(log_price, order=(3,0,0)) #### Cambiar ACA segun resultado del autoarima
arima_fit = model.fit()
print(arima_fit.summary())


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA



steps = 30

fc = arima_fit.get_forecast(steps=steps)

fc_mean_log = fc.predicted_mean
fc_ci_log = fc.conf_int()

# 1) Crear índice FUTURO (días hábiles)
future_idx = pd.bdate_range(start=log_price.index[-1] + pd.Timedelta(days=1), periods=steps)

# 2) Volver a precio + poner índice futuro
fc_price = pd.Series(np.exp(fc_mean_log.values), index=future_idx, name="fc_price")
lower    = pd.Series(np.exp(fc_ci_log.iloc[:,0].values), index=future_idx, name="lower")
upper    = pd.Series(np.exp(fc_ci_log.iloc[:,1].values), index=future_idx, name="upper")

forecast_df = pd.concat([fc_price, lower, upper], axis=1)
print(forecast_df.head())

# 3) Plot correcto (todo con fechas)
plt.figure(figsize=(12,5))
plt.plot(price, label="Precio histórico")
plt.plot(fc_price, label="Forecast precio")
plt.fill_between(future_idx, lower, upper, alpha=0.2, label="IC 95%")
plt.title(f"Forecast de Precio — {ticker} (ARIMA en log_price)")
plt.xlabel("Fecha"); plt.ylabel("Precio")
plt.legend()
plt.show()


